In [79]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import pyplot
import math
import random as ran
from scipy.stats import truncnorm

In [80]:
###real data 임시생성
size=20
a=[0]*size
for i in range(size):
    r = [ran.random() for i in range(500)]
    s = sum(r)
    r = [ i/s for i in r ]
    a[i]=r
a=np.array(a)

In [81]:
total_epoch = 100
batch_size = 5
learning_rate = 0.0002

n_hidden = 256
n_input = 500 * 1
n_noise = 1

In [82]:
X = tf.placeholder(tf.float32)
Z = tf.placeholder(tf.float32, [None, n_noise])

G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.5))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.1))
G_b2 = tf.Variable(tf.zeros([n_input]))
s=tf.Variable(0.)


In [83]:
def generator(noise_z):
    hidden = tf.nn.relu(
                    tf.matmul(noise_z, G_W1) + G_b1)
    out = tf.nn.relu(
                    tf.matmul(hidden, G_W2) + G_b2)
    
# 생성된 값을 0.3(new_min)~0.6(new_max) 값으로 반환    
    ma=tf.reduce_max(out)
    mi=tf.reduce_min(out)

    out2 = tf.div(
       tf.subtract(
          out, mi
       ), 
       tf.subtract(
          ma,mi
       ))

    output=tf.add(tf.multiply(tf.constant(0.3,shape=[1,500]),out2),tf.constant(0.3,shape=[1,500]))
    return output


In [84]:
def discriminator(inputs):
    
# 기준 값a[0]과 비슷하게 출력
    arg = tf.convert_to_tensor(a[0], dtype=tf.float32)
    out=tf.subtract(inputs,arg)
    out2= tf.reduce_sum(out)


# sum w_i = 1이 최대한 되도록!
    out=tf.reduce_sum(inputs)
    k=tf.subtract(out,1.)
    output=tf.cond(k<0,lambda:-k,lambda:k)
    

    return output

In [85]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [86]:
G = generator(Z)
loss_G = discriminator(G)

saver = tf.train.Saver()
G_var_list = [G_W1, G_b1, G_W2, G_b2,s]

train_G = tf.train.AdamOptimizer(learning_rate).minimize(loss_G,
                                                       var_list=G_var_list)


In [87]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(size/batch_size)
loss_val_G = 0

for epoch in range(total_epoch):

    
# 자산 갯수 중 100개만 선택해라 !
    s=-100
    noise = get_noise(1, n_noise)
    dfd= sess.run(G, feed_dict={Z: noise})
    for j in range(500):
        if dfd[0][j]>0:
            s=s+1
    if s<0 : s=-s
    else: s=s

    loss_G = discriminator(G)+s   
    _, loss_val_G = sess.run([train_G, loss_G],
                             feed_dict={Z: noise})
    val.append(loss_val_G)
    
    #save_path = saver.save(sess, "./model.ckpt")


